In [17]:
!pip install --upgrade pip
!pip uninstall -y shimmy stable-baselines3 gymnasium
!pip install gymnasium==0.29.1
!pip install shimmy
!pip install stable-baselines3[extra]

Defaulting to user installation because normal site-packages is not writeable
Found existing installation: Shimmy 2.0.0
Uninstalling Shimmy-2.0.0:
  Successfully uninstalled Shimmy-2.0.0
Found existing installation: stable-baselines3 1.7.0
Uninstalling stable-baselines3-1.7.0:
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/lib/python3.10/shutil.py", line 816, in move
    os.rename(src, real_dst)
PermissionError: [Errno 13] Permission denied: '/usr/local/lib/python3.10/dist-packages/stable_baselines3-1.7.0-py3.10.egg' -> '/tmp/pip-uninstall-ludb2olx'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/sohrab/.local/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "/home/sohrab/.local/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "/home/sohrab/.lo

In [18]:
%matplotlib inline
import matplotlib
from matplotlib import rc
rc('animation', html='jshtml')

In [19]:
import numpy as np
from gymnasium import Env
from gymnasium.spaces import Tuple, Box, Discrete
from stable_baselines3.common import env_checker

import random
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [20]:
def hec(psic,psid):
    HE = psic - psid
    if HE>1 :
      HE = HE - 2
    if HE<-1 :
      HE = HE + 2
    return HE

def dist(x1,y1,x2,y2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)

In [21]:
class Agent1() :

    def __init__(self,xi, yi, psii, vi, dt) :
        self.x = [xi]
        self.y = [yi]
        self.psi = [psii]
        self.v = [vi]
        self.dt = dt

        self.polars = np.full((30,),0.0)


    def kinematics(self,action,collision):
        v_a = self.v[-1] + action[0]*0.1*self.dt
        if v_a<0.2:
          v_a=0.2
        if v_a>1:
          v_a = 1
        if collision == True and v_a>0.5:
          v_a = 0.5

        psi_a = self.psi[-1] + action[1]*0.1*self.dt
        if psi_a<-1:
          psi_a= psi_a +2
        if psi_a>1:
          psi_a = psi_a - 2

        x_a = self.x[-1] + 10*v_a*np.cos(np.pi*psi_a)*self.dt
        y_a = self.y[-1] + 10*v_a*np.sin(np.pi*psi_a)*self.dt

        self.x.append(x_a)
        self.y.append(y_a)
        self.psi.append(psi_a)
        self.v.append(v_a)

    def radar(self, list_obs, a):
        k = 0
        self.polars = np.full((30,),0.0)
        angle = np.arange(self.psi[-1]-2/3,self.psi[-1]+2/3+0.01,4/15)
        for i in range(5):
            for j in angle:
                x_point = self.x[-1] + (i+1)*a*np.cos(j*np.pi)
                y_point = self.y[-1] + (i+1)*a*np.sin(j*np.pi)
                for obstacle in list_obs :
                    if dist(x_point, y_point, obstacle.x[-1], obstacle.y[-1])<= obstacle.r :
                      self.polars[k] = 1
                      break

                k = k+1

In [22]:
class Obstacle():
    def __init__(self,xf,yf,dt,rad=20):
        self.r = rad
        self.x = [xf/2]
        self.y = [yf/2]
        self.v = np.random.randint(2,5)
        self.direction = np.random.uniform(-np.pi,np.pi)
        self.dt = dt

    def kinematics(self):
        x_a = self.x[-1] + self.v*np.cos(self.direction)*self.dt
        y_a = self.y[-1] + self.v*np.sin(self.direction)*self.dt
        self.direction = self.direction + np.random.uniform(-np.pi/6,np.pi/6)
        self.x.append(x_a)
        self.y.append(y_a)

In [23]:
def Reward(HE,v,polars,collision):

    a = (-1)* abs(HE)

    b = 0
    radial_weights = [5,3,1.2,0.5,0.2]
    k = 0
    for i in range(5): #dis
        for j in range(6): #angle
            if polars[k]==1 :
                b-=radial_weights[i]/60
            k+=1

    c = min(a,b)*v

    d = 0
    if collision :
        d = -1

    return 0.5*a + 0.1*b + 0.13*c+ 0.2*d + 0.15*v

In [24]:
from torch.nn.parallel.scatter_gather import T
class CustomEnv(Env):
    xfp = 0
    yfp = 0
    xp = []
    yp = []
    vp = []
    lis = []


    metadata = {"render.modes": ["human"]}
    i = 0


    def __init__(self):
        super().__init__()

        self.psi0 = np.random.uniform(-np.pi,np.pi)
        self.xf = 500 * np.cos(self.psi0)
        self.yf = 500 * np.sin(self.psi0)
        self.del_t = 0.25
        self.steps= 0

        self.agent = Agent1(0,0,self.psi0,0.2,self.del_t)

        self.list_obs = []
        self.n_obs = np.random.randint(3,5)
        for i in range(self.n_obs) :
            obstacle = Obstacle((0.6*random.random()+0.2)*self.xf,
                                (0.6*random.random()+0.2)*self.yf,
                                self.del_t, 30)
            self.list_obs.append(obstacle)

        self.agent.radar(self.list_obs, 6)

        self.collision = False
        for obstacle in self.list_obs :
            if dist(self.agent.x[-1],self.agent.y[-1], obstacle.x[-1], obstacle.y[-1])<=obstacle.r :
              self.collision = True
              break

        self.done = False
        self.cum_rew = []
        self.epi_rew = 0


        self.he = hec(self.agent.psi[-1], np.arctan2(self.yf-self.agent.y[-1], self.xf-self.agent.x[-1])/np.pi)
        self.action_space = Box(low = np.array([-1,-1]), high = np.array([1,1]), shape = (2,), dtype = np.float32)
        low = np.full((30,),0.0)
        low = np.append(low,[-1,-1])
        high = np.full((32,),1.0)
        self.observation_space = Box(low=low, high=high, shape = (32,), dtype=np.float32)


    def step(self, action):

        self.agent.kinematics(action, self.collision)
        for obstacle in self.list_obs :
            obstacle.kinematics()

        self.agent.radar(self.list_obs, 10)

        self.collision = False
        for obstacle in self.list_obs :
            if dist(self.agent.x[-1],self.agent.y[-1], obstacle.x[-1], obstacle.y[-1])<=obstacle.r :
              self.collision = True
              break



        self.he = hec(self.agent.psi[-1], np.arctan2(self.yf-self.agent.y[-1], self.xf-self.agent.x[-1])/np.pi)
        self.observation = np.array(np.append(self.agent.polars, [self.he, self.agent.v[-1]]),dtype = np.float32)

        self.reward = Reward(self.he, self.agent.v[-1],self.agent.polars,self.collision)
        self.epi_rew = self.reward + self.epi_rew

        self.steps= self.steps + self.del_t

        if dist(self.agent.x[-1],self.agent.y[-1],self.xf,self.yf) < 15:
            self.done = True
            self.render()


        if self.steps>= 70 :
            self.done = True
            self.render()

        if self.done == True:
            self.cum_rew = np.append(self.cum_rew,self.epi_rew)

        self.info = {}
        return self.observation, self.reward, self.done,False, self.info

    def reset(self, seed=None, options=None):

        super().reset(seed=seed)

        self.i+=1
        self.psi0 = np.random.uniform(-np.pi,np.pi)
        self.xf = 500 * np.cos(self.psi0)
        self.yf = 500 * np.sin(self.psi0)
        self.del_t = 0.25
        self.steps= 0

        self.agent = Agent1(0,0,self.psi0/np.pi,0.2,self.del_t)

        self.list_obs = []
        self.n_obs = np.random.randint(3,5)
        for i in range(self.n_obs) :
            obstacle = obstacle = Obstacle((0.6*random.random()+0.2)*self.xf,
                                          (0.6*random.random()+0.2)*self.yf,
                                          self.del_t, 30)
            self.list_obs.append(obstacle)

        self.agent.radar(self.list_obs, 10)

        self.collision = False
        for obstacle in self.list_obs :
            if dist(self.agent.x[-1],self.agent.y[-1], obstacle.x[-1], obstacle.y[-1])<=obstacle.r :
              self.collision = True
              break

        self.epi_rew = 0

        self.he = hec(self.agent.psi[-1], np.arctan2(self.yf-self.agent.y[-1], self.xf-self.agent.x[-1])/np.pi)

        self.done = False

        observation = np.array(np.append(self.agent.polars, [self.he, self.agent.v[-1]]),dtype = np.float32)
        info = {}
        return observation, info

    def render(self):
        self.xp = self.agent.x
        self.yp = self.agent.y
        self.vp = self.agent.v
        self.xfp = self.xf
        self.yfp = self.yf
        self.lis = self.list_obs

    def close(self):
        ...

In [25]:
def visualize(env):
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.set_xlim(( min(0,env.xfp)-40, max(0,env.xfp)+40))
    ax.set_ylim(( min(0,env.yfp)-40, max(0,env.yfp)+40))
    ax.plot(env.xfp,env.yfp,'b.', ms = 3)


    def update(frame):
        ax.clear()

        ax.set_aspect('equal')
        ax.set_xlim(( min(0,env.xfp)-40, max(0,env.xfp)+40))
        ax.set_ylim(( min(0,env.yfp)-40, max(0,env.yfp)+40))
        ax.plot(env.xfp,env.yfp,'r*', ms = 8)
        ax.set_title(f'Velocity: {env.vp[frame-1]}')


        # draw circle around a point
        for obstacle in env.lis :
            Drawing_colored_circle = plt.Circle(( obstacle.x[frame-1], obstacle.y[frame-1]), obstacle.r, color='red', alpha=0.4)
            ax.add_artist( Drawing_colored_circle )

        # plot path follwed by ship
        pt = ax.plot(env.xp[:frame], env.yp[:frame], color= 'black') # path of ship
        ax.scatter(env.xp[frame-1],env.yp[frame-1],marker='>',color='black' ) # ship position

        ax.set_aspect('equal')
        ax.grid(True)


    return animation.FuncAnimation(fig, update, frames = len(env.xp), interval=50, repeat=False)

In [26]:
env = CustomEnv()

In [27]:
env_checker.check_env(env)

AssertionError: Your environment must inherit from the gym.Env class cf https://github.com/openai/gym/blob/master/gym/core.py

In [ ]:
from stable_baselines3 import DDPG
policy_kwargs = dict(net_arch=[128,128])

model = DDPG("MlpPolicy", env,policy_kwargs = policy_kwargs, verbose=1)


# obs = env.reset()
# while True:
#     action, _states = model.predict(obs, deterministic=True)
#     obs, reward, done, info = env.step(action)
#     if done:
#         obs = env.reset()
#         break

In [ ]:
obs,info = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done,trun, info = env.step(action)
    if done:
        break

anim = visualize(env)
anim

In [ ]:
writervideo = animation.FFMpegWriter(fps=40)
anim.save('vd_final.mp4', writer=writervideo)

In [ ]:
model.save("DDPG_sb3")
Episode_rewards = env.cum_rew
plt.figure(figsize=(9,6))
running_avg = np.empty(len(Episode_rewards))
for t in range(len(Episode_rewards)):
        running_avg[t] = np.mean(Episode_rewards[max(0, t-100):(t+1)])
plt.plot(Episode_rewards[:600],color="teal",alpha=0.2)
plt.plot(running_avg,color="teal",linewidth=2.0)
plt.ylabel("Reward Units")
plt.xlabel("No. of Episode")
plt.title("Deep Deterministic Policy Gradient (DDPG)")
plt.grid()
plt.savefig("DDPG_Rewards_waves_Nomoto.jpg",dpi = 420)
plt.show()